# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fad740b7dc0>
├── 'a' --> tensor([[-0.3867, -0.4502, -1.7339],
│                   [-0.5838,  1.1508,  1.1885]])
└── 'x' --> <FastTreeValue 0x7fad74058490>
    └── 'c' --> tensor([[ 1.2505,  0.0467, -1.3784,  0.0556],
                        [-0.2387,  0.4657,  1.3429,  1.3016],
                        [ 1.0546,  0.4774, -1.3144,  0.4751]])

In [4]:
t.a

tensor([[-0.3867, -0.4502, -1.7339],
        [-0.5838,  1.1508,  1.1885]])

In [5]:
%timeit t.a

61.1 ns ± 0.0341 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fad740b7dc0>
├── 'a' --> tensor([[ 0.4586, -0.4561,  1.1784],
│                   [-0.4969, -1.6781,  1.3396]])
└── 'x' --> <FastTreeValue 0x7fad74058490>
    └── 'c' --> tensor([[ 1.2505,  0.0467, -1.3784,  0.0556],
                        [-0.2387,  0.4657,  1.3429,  1.3016],
                        [ 1.0546,  0.4774, -1.3144,  0.4751]])

In [7]:
%timeit t.a = new_value

61 ns ± 0.0454 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3867, -0.4502, -1.7339],
               [-0.5838,  1.1508,  1.1885]]),
    x: Batch(
           c: tensor([[ 1.2505,  0.0467, -1.3784,  0.0556],
                      [-0.2387,  0.4657,  1.3429,  1.3016],
                      [ 1.0546,  0.4774, -1.3144,  0.4751]]),
       ),
)

In [10]:
b.a

tensor([[-0.3867, -0.4502, -1.7339],
        [-0.5838,  1.1508,  1.1885]])

In [11]:
%timeit b.a

52.6 ns ± 0.0839 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7735, -1.0798, -0.5141],
               [-1.0147, -1.1593,  0.3065]]),
    x: Batch(
           c: tensor([[ 1.2505,  0.0467, -1.3784,  0.0556],
                      [-0.2387,  0.4657,  1.3429,  1.3016],
                      [ 1.0546,  0.4774, -1.3144,  0.4751]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

848 ns ± 5.48 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 14.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

134 µs ± 443 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 364 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7facd09c3730>
├── 'a' --> tensor([[[-0.3867, -0.4502, -1.7339],
│                    [-0.5838,  1.1508,  1.1885]],
│           
│                   [[-0.3867, -0.4502, -1.7339],
│                    [-0.5838,  1.1508,  1.1885]],
│           
│                   [[-0.3867, -0.4502, -1.7339],
│                    [-0.5838,  1.1508,  1.1885]],
│           
│                   [[-0.3867, -0.4502, -1.7339],
│                    [-0.5838,  1.1508,  1.1885]],
│           
│                   [[-0.3867, -0.4502, -1.7339],
│                    [-0.5838,  1.1508,  1.1885]],
│           
│                   [[-0.3867, -0.4502, -1.7339],
│                    [-0.5838,  1.1508,  1.1885]],
│           
│                   [[-0.3867, -0.4502, -1.7339],
│                    [-0.5838,  1.1508,  1.1885]],
│           
│                   [[-0.3867, -0.4502, -1.7339],
│                    [-0.5838,  1.1508,  1.1885]]])
└── 'x' --> <FastTreeValue 0x7facd09cdfa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.7 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7facd0933280>
├── 'a' --> tensor([[-0.3867, -0.4502, -1.7339],
│                   [-0.5838,  1.1508,  1.1885],
│                   [-0.3867, -0.4502, -1.7339],
│                   [-0.5838,  1.1508,  1.1885],
│                   [-0.3867, -0.4502, -1.7339],
│                   [-0.5838,  1.1508,  1.1885],
│                   [-0.3867, -0.4502, -1.7339],
│                   [-0.5838,  1.1508,  1.1885],
│                   [-0.3867, -0.4502, -1.7339],
│                   [-0.5838,  1.1508,  1.1885],
│                   [-0.3867, -0.4502, -1.7339],
│                   [-0.5838,  1.1508,  1.1885],
│                   [-0.3867, -0.4502, -1.7339],
│                   [-0.5838,  1.1508,  1.1885],
│                   [-0.3867, -0.4502, -1.7339],
│                   [-0.5838,  1.1508,  1.1885]])
└── 'x' --> <FastTreeValue 0x7fad7405fbe0>
    └── 'c' --> tensor([[ 1.2505,  0.0467, -1.3784,  0.0556],
                        [-0.2387,  0.4657,  1.3429,  1.3016],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 85.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.2 µs ± 113 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.2505,  0.0467, -1.3784,  0.0556],
                       [-0.2387,  0.4657,  1.3429,  1.3016],
                       [ 1.0546,  0.4774, -1.3144,  0.4751]],
              
                      [[ 1.2505,  0.0467, -1.3784,  0.0556],
                       [-0.2387,  0.4657,  1.3429,  1.3016],
                       [ 1.0546,  0.4774, -1.3144,  0.4751]],
              
                      [[ 1.2505,  0.0467, -1.3784,  0.0556],
                       [-0.2387,  0.4657,  1.3429,  1.3016],
                       [ 1.0546,  0.4774, -1.3144,  0.4751]],
              
                      [[ 1.2505,  0.0467, -1.3784,  0.0556],
                       [-0.2387,  0.4657,  1.3429,  1.3016],
                       [ 1.0546,  0.4774, -1.3144,  0.4751]],
              
                      [[ 1.2505,  0.0467, -1.3784,  0.0556],
                       [-0.2387,  0.4657,  1.3429,  1.3016],
                       [ 1.0546,  0.4774, -1.3144,  0.4751]],

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 162 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.2505,  0.0467, -1.3784,  0.0556],
                      [-0.2387,  0.4657,  1.3429,  1.3016],
                      [ 1.0546,  0.4774, -1.3144,  0.4751],
                      [ 1.2505,  0.0467, -1.3784,  0.0556],
                      [-0.2387,  0.4657,  1.3429,  1.3016],
                      [ 1.0546,  0.4774, -1.3144,  0.4751],
                      [ 1.2505,  0.0467, -1.3784,  0.0556],
                      [-0.2387,  0.4657,  1.3429,  1.3016],
                      [ 1.0546,  0.4774, -1.3144,  0.4751],
                      [ 1.2505,  0.0467, -1.3784,  0.0556],
                      [-0.2387,  0.4657,  1.3429,  1.3016],
                      [ 1.0546,  0.4774, -1.3144,  0.4751],
                      [ 1.2505,  0.0467, -1.3784,  0.0556],
                      [-0.2387,  0.4657,  1.3429,  1.3016],
                      [ 1.0546,  0.4774, -1.3144,  0.4751],
                      [ 1.2505,  0.0467, -1.3784,  0.0556],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 190 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 13.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
